In [1]:
!pip install -U datasets
!pip install -U evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch

## Instruction-Finetuned GPT-2

Probando GPT-2 355M entrenado en los datasets ANLI, BoolQ, Common Gen y XSum. Todos son diferentes tareas de PLN, y cada dataset fue formateado a instrucciones. Por lo tanto, este enfoque es conocido como multi-task instruction-tuning.

Se entrenó el modelo durante 3 épocas en una RTX 3050 8GB usando una taza de aprendizaje de 5e-4. El entrenamiento duró 6 horas.

Ver el código fuente en [Github](https://github.com/oscarnavmac/flan-gpt2/tree/main).

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = "OscarNav/gpt2-multitask"
print(device)

cuda


In [4]:
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-medium")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Evaluacion Cualitativa

Primero, necesitamos clonar el repositorio

In [5]:
!git clone https://github.com/oscarnavmac/flan-gpt2.git
%cd flan-gpt2

Cloning into 'flan-gpt2'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 55 (delta 27), reused 44 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 9.93 KiB | 9.93 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/flan-gpt2


In [6]:
from eval_utils import Evaluation, generate
from templates import PATTERNS
from data_utils import format_example

eval = Evaluation(model, tokenizer, device)

#### ANLI

In [7]:
dataset = load_dataset("facebook/anli", split="test_r1")
int2str = dataset.features['label'].int2str
dataset = dataset.map(lambda example: {"answer": int2str(example["label"])})

README.md:   0%|          | 0.00/7.98k [00:00<?, ?B/s]

train_r1-00000-of-00001.parquet:   0%|          | 0.00/3.14M [00:00<?, ?B/s]

dev_r1-00000-of-00001.parquet:   0%|          | 0.00/351k [00:00<?, ?B/s]

test_r1-00000-of-00001.parquet:   0%|          | 0.00/353k [00:00<?, ?B/s]

train_r2-00000-of-00001.parquet:   0%|          | 0.00/6.53M [00:00<?, ?B/s]

...

Generating test_r2 split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_r3 split:   0%|          | 0/100459 [00:00<?, ? examples/s]

Generating dev_r3 split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating test_r3 split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
example = dataset[0]

In [9]:
prompt, ground_truth = format_example(example, PATTERNS["anli"], 0).values()

In [10]:
print(prompt)

Ernest Jones is a British jeweller and watchmaker. Established in 1949, its first store was opened in Oxford Street, London. Ernest Jones specialises in diamonds and watches, stocking brands such as Gucci and Emporio Armani. Ernest Jones is part of the Signet Jewelers group.

Based on the paragraph above can we conclude that "The first Ernest Jones store was opened on the continent of Europe."?


In [11]:
prediction = generate(model, tokenizer, device, [prompt])
print("\n",prediction[0])

Evaluating model: 100%|██████████| 1/1 [00:02<00:00,  2.41s/it]


 neutral neutral neutral neutral contradiction contradiction. neutral contradiction neutral contradiction. contradiction or neutral neutral contradiction. neutral contradiction neutral contradiction neutral contradiction entailment neutral entailment neutral contradiction. neutral contradiction entailment neutral or neutral entail


In [12]:
print(ground_truth)

entailment


Vemos que al modelo no le va muy bien en esta tarea. Además hay que recordar que el limite de tokens que pusimos fue de 40, lo cual no seria adecuado para esta tarea. Veamos con las demás...

#### Common Gen

In [13]:
dashline = '-'.join('' for x in range(100))

In [14]:
dataset = load_dataset('allenai/common_gen', split='validation')
example = dataset[0]
prompt, ground_truth = format_example(example, PATTERNS["common_gen"], 0).values()
prediction = generate(model, tokenizer, device, [prompt])[0]
print(dashline)
print(f'INPUT PROMPT \n{prompt}')
print(dashline)
print(f"Ground Truth: \n\n{ground_truth}")
print(dashline)
print(f'MODEL GENERATION: \n\n{prediction}')

README.md:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/171k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/31.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67389 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4018 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1497 [00:00<?, ? examples/s]

Evaluating model: 100%|██████████| 1/1 [00:00<00:00,  1.41it/s]

---------------------------------------------------------------------------------------------------
INPUT PROMPT 
Concepts: ['field', 'look', 'stand']

Write a sentence that includes all these words.
---------------------------------------------------------------------------------------------------
Ground Truth: 

The player stood in the field looking at the batter.
---------------------------------------------------------------------------------------------------
MODEL GENERATION: 

A man that is standing in wheat field outside looking towards a camera.


Esta vez mejoró, aunque no le salió muy bien fue capaz de entender la tarea.
Algo muy interesante a notar es que el modelo acortó la generación a diferencia de la tarea anterior, es decir, no generó los 40 tokens forzosamente.

Otro punto clave es que el modelo no ha sufrido de un olvidamiento catastrófico. Aunque claro que solo lo entrenamos en 4 tareas.

#### BoolQ

In [15]:
dataset = load_dataset('google/boolq', split='validation')
example = dataset[0]
prompt, ground_truth = format_example(example, PATTERNS["bool_q"], 0).values()
prediction = generate(model, tokenizer, device, [prompt])[0]
print(dashline)
print(f'INPUT PROMPT \n{prompt}')
print(dashline)
print(f"Ground Truth: \n\n{ground_truth}")
print(dashline)
print(f'MODEL GENERATION: \n\n{prediction}')

README.md:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.69M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

Evaluating model: 100%|██████████| 1/1 [00:00<00:00,  1.13it/s]

---------------------------------------------------------------------------------------------------
INPUT PROMPT 
All biomass goes through at least some of these steps: it needs to be grown, collected, dried, fermented, distilled, and burned. All of these steps require resources and an infrastructure. The total amount of energy input into the process compared to the energy released by burning the resulting ethanol fuel is known as the energy balance (or ``energy returned on energy invested''). Figures compiled in a 2007 report by National Geographic Magazine point to modest results for corn ethanol produced in the US: one unit of fossil-fuel energy is required to create 1.3 energy units from the resulting ethanol. The energy balance for sugarcane ethanol produced in Brazil is more favorable, with one unit of fossil-fuel energy required to create 8 from the ethanol. Energy balance estimates are not easily produced, thus numerous such reports have been generated that are contradictory. F

Aquí pasó al revés que con el BoolQ, el modelo respondió bien (bueno, más o menos) pero no entendió muy bien la tarea, o sea, respondió Falso pero siguió generando más texto innecesario.

#### XSum

In [16]:
dataset = load_dataset('EdinburghNLP/xsum', split='test')
example = dataset[0]
prompt, ground_truth = format_example(example, PATTERNS["xsum"], 0).values()
prediction = generate(model, tokenizer, device, [prompt])[0]
#print(dashline)
print(f'INPUT PROMPT \n{prompt}')
print(dashline)
print(f"Ground Truth: \n\n{ground_truth}")
print(dashline)
print(f'MODEL GENERATION: \n\n{prediction}')

0000.parquet:   0%|          | 0.00/304M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/17.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Evaluating model: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]

INPUT PROMPT 
Summarize:

Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.
Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.
However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.
Andrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, sa

Más o menos, parece que medio logró entender la tarea pero le costó hacer un resumen conciso y preciso.

Lo bueno es que en ningún momentó cruzó las tareas, entonces significa que GPT-2 definitivamente puede con 4 tareas sin requerir destilación.

### Evaluacion Cuantitativa

Para no hacer el cuento largo, usemos métricas para evaluar cada dataset. Para ANLI y BooQ usaremos precisión porque ambas son tareas de clasificación. Para XSum y Common Gen usaremos ROUGE. (para más detalles revisar el repo)

Para cada dataset solo evaluaremos los primeros 500 ejemplos. Para una evaluación más precisa dejar este argumento en default para evaluar en todo el dataset, aunque esto llevaría más horas.

In [17]:
res_anli = eval.anli(500)
res_boolq = eval.bool_q(500)
res_commongen = eval.common_gen(500)
res_xsum = eval.xsum(500)

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Evaluating model: 100%|██████████| 500/500 [05:53<00:00,  1.42it/s]


Filter:   0%|          | 0/3270 [00:00<?, ? examples/s]

Evaluating model: 100%|██████████| 500/500 [05:46<00:00,  1.44it/s]


Filter:   0%|          | 0/4018 [00:00<?, ? examples/s]

Evaluating model: 100%|██████████| 500/500 [05:38<00:00,  1.47it/s]


Filter:   0%|          | 0/11334 [00:00<?, ? examples/s]

Evaluating model: 100%|██████████| 368/368 [04:30<00:00,  1.36it/s]


In [18]:
print(f"Total accuracy on ANLI is {res_anli}")
print(f"Total accuracy on BoolQ is {res_boolq}")
print(f"Rouge-1 score on Common Gen is {res_commongen}")
print(f"Rouge-LSum score on XSum is {res_xsum}")

Total accuracy on ANLI is 0.33
Total accuracy on BoolQ is 0.588
Rouge-1 score on Common Gen is 0.25293517014722844
Rouge-LSum score on XSum is 0.14147674372793262


Los resultados no son tan buenos aún, quizás haga falta entrenar por más tiempo o usar un modelo más grande. Aún falta también experimentar con destilación.